<a href="https://colab.research.google.com/github/Whereamiactually/lyceumcompling11/blob/main/Crawlers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обкачивание интернета
(Основано на тетрадке Тани Казаковой.)

Сегодня мы немного посмотрим на модуль [requests](https://realpython.com/python-requests/), который позволят выгружать .html странички с просторов Интернета.

Конечно, есть специальный [учебник](https://www.w3schools.com). Там не только про работу с .html, но ещё есть и про многие другие базовые вещи.


In [1]:
import requests

In [ ]:
response = requests.get("https://www.moscowzoo.ru/about-zoo/news/novosti-zooparka/v-moskovskom-zooparke-poselilis-marabu/", verify = False)

В `response` теперь лежит ответ сервера. В нем не просто содержится html-код страницы, но еще и информация о том, как все прошло.

In [ ]:
response

Вот список возможных кодов с их приблизительными значениями:

* 1XX — информация,
* 2XX — успешно,
* 3XX — перенаправление,
* 4XX — ошибка клиента (ошибка на вашей стороне),
* 5XX — ошибка сервера (ошибка на их стороне).

In [ ]:
response.status_code

Можем попросить выдать заголовки, где содержится техническая информация о нашем запросе и о содержании страницы.

Обычно требуется заголовок Content-Type, поскольку он показывает формат данных, например HTML, JSON, PDF, обычный текст и т. д.

In [5]:
response.headers

{'Server': 'nginx/1.24.0', 'Date': 'Fri, 27 Oct 2023 08:40:27 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'P3P': 'policyref="/bitrix/p3p.xml", CP="NON DSP COR CUR ADM DEV PSA PSD OUR UNR BUS UNI COM NAV INT DEM STA"', 'X-Powered-CMS': 'Bitrix Site Manager (fbee37191be309d4ddfe59f62a792d4f)', 'Set-Cookie': 'PHPSESSID=9e42e02975db56f2bcc1673695f9ae2c; path=/; HttpOnly', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate', 'Pragma': 'no-cache', 'Content-Encoding': 'gzip'}

In [ ]:
# выводим html-код страницы
print(response.text)

Если сайт защищается от краулеров на Python'e, можно представиться законным браузером, например, Мозиллой.

In [ ]:
url = 'https://www.moscowzoo.ru/about-zoo/news/novosti-zooparka/v-moskovskom-zooparke-poselilis-marabu/'  # адрес страницы, которую мы хотим скачать
user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'  # хотим притворяться браузером

response = requests.get(url, headers = {'User-Agent': user_agent}, verify = False)

Или использовать специальную библиотеку.

In [ ]:
!pip install fake-useragent

In [8]:
from fake_useragent import UserAgent

In [ ]:
user_agent = UserAgent().chrome # хотим притворяться браузером
user_agent

In [ ]:
response = requests.get("https://www.moscowzoo.ru/about-zoo/news/novosti-zooparka/v-moskovskom-zooparke-poselilis-marabu/",
                        headers={'User-Agent': user_agent},
                        verify = False)

Также `response` содержит ссылку, по которой мы получили данные. Если наш запрос был перенаправлен с одного сайта на другой, то ссылки в запросе и в выдаче будут отличаться.

In [ ]:
response.url

Чтобы достать что-то из определенного тега (например, заголовок), можно использовать специальные библиотеки Python'a, например, **BeautifulSoup** или **lxml**.


In [ ]:
pip install bs4

In [16]:
from bs4 import BeautifulSoup

Вначале нам нужно создать суп из полученного html-кода страницы.

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser') # надо прописать парсер
print(soup.prettify())  # печатаем приукрашенный суп

По виду этот суп почти не отличается от того, что мы видели до этого. Но теперь мы можем искать напрямую по названию тегов то, что нам нужно.

Например, если мы хотим вывести заголовок, нужно посмотреть, какими тегами он оформляется. На сайте Московского зоопарка заголовки лежат внутри тегов `cp-banner`. Так и найдем.

In [ ]:
name = soup.find('cp-banner')
print(name.prettify())

А внутри этого раздела текст заголовка лежит внутри первого тега `template`.

In [ ]:
name2 = name.find('template')
print(name2.prettify())

Нас интересует только текст, поэтому используем `get_text`.

In [ ]:
print(name2.get_text())

Если бы мы изначально искали по `template`, так бы не сработало, потому что такой тег есть еще в некоторых местах. То есть вначале нам нужно однозначно определить теги, которые соответствуют **только** заголовкам.

Но можно ту же самую операцию выполнить и за одну строку года. Следующий код ищет теги `template` только внутри тегов `cp-banner`.

In [24]:
soup.select('cp-banner > template')

[<template #title="">В Московском зоопарке поселились марабу</template>,
 <template #description=""></template>]

In [ ]:
# заголовок находится в первом из найденных тегов,
# поэтому мы можем его вывести
print(soup.select('cp-banner > template')[0])

# также мы можем вывести его содержание
print('Содержание тега:')
print(soup.select('cp-banner > template')[0].get_text())

<template #title="">ПЕРВЫЙ В РОССИИ ДЕТЕНЫШ БОЛЬШОЙ ПАНДЫ</template>
Содержание тега:
ПЕРВЫЙ В РОССИИ ДЕТЕНЫШ БОЛЬШОЙ ПАНДЫ


А теперь найдём текст новости.

В каком теге он лежит?

Он лежит в теге `<div class="dp-content-inner" style="color:#fff;">`

Так как тегов `div` очень много, то нам нужно специфицировать атрибуты, по которым мы сможем однозначно определить местоположения текста новости.

А именно: `class: "dp-content-inner"`.

In [ ]:
post = soup.find('div', {'class':"dp-content-inner"})  # вот так можно указать атрибут тега
print(post.prettify())

In [ ]:
text = post.get_text()  # получаем текст
print(text)

Вот здесь можно [посмотреть часть документации BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html).

Теперь с этим текстом можно делать все то, что мы проходили ранее: искать именованные сущности, проводить морфологический анализ, определять тему текста и пр.

И все это мы можем потом вернуть пользователю, если он попросит.

Но теперь, одна новость - это, конечно, хорошо, но нам бы хотелось побольше... Как нам это сделать?

Можем зайти на сайт и посмотреть его структуру. Также мы можем скачать html-код страницы с новостями.

In [ ]:
response = requests.get("https://www.moscowzoo.ru/about-zoo/news/novosti-zooparka/",
                        headers={'User-Agent': user_agent},
                        verify = False)

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser') # надо прописать парсер
print(soup.prettify())

Так мы можем скачать все, что находится в нужных нам тегах.

In [ ]:
post = soup.find_all('div', {'class':"article-item"})
print(post)

Теперь вытянем из каждой новости ссылку на ее отдельную страницу, аналогичную той, которую мы уже видели.

Ссылки содержатся в теге `a` в его атрибуте `href`. Достанем их!

In [43]:
all_links = []
for news in post:
  link = news.find('a', class_='article-item-link').get('href')
  all_links.append(link)

In [ ]:
print(all_links)

Теперь перейдем по каждой ссылке и выкачаем из нее заголовок и текст новости!

In [ ]:
headers_news = {}
host = "https://www.moscowzoo.ru" # нам нужно отдельно задать адрес главного сайта
for link in all_links:
  whole_link = host + link
  print(whole_link)
  news_item = requests.get(whole_link, headers={'User-Agent': user_agent}, verify = False)
  soup = BeautifulSoup(news_item.text, 'html.parser')
  header = soup.select('cp-banner > template')[0].get_text()
  headers_news[header] = soup.find('div', {'class':"dp-content-inner"}).get_text()

In [55]:
import re

In [59]:
for key, value in headers_news.items():
  headers_news[key] = re.sub('[\n\r\t]', '', value).strip()

In [ ]:
headers_news

Хотим еще больше данных!

Тогда давайте смотреть на все страницы...

Вначале посмотрим, как оформляется ссылки на разные страницы.

Ага! Значит, она выглядит вот так: `https://www.moscowzoo.ru/about-zoo/news/novosti-zooparka/?PAGEN_1=X`, где X это номер страницы.

Значит, мы можем завести переменную, которая с каждой новой страницей увеличивается на один и подставляется в ссылку.

Давайте выведем новости с первых двух страниц.

In [ ]:
headers_news_2_pages = {}
main_link = "https://www.moscowzoo.ru/about-zoo/news/novosti-zooparka/?PAGEN_1="
for i in range(1, 3):
  print(main_link + str(i))
  response = requests.get(main_link + str(i),
                          headers={'User-Agent': user_agent},
                          verify = False)
  soup = BeautifulSoup(response.text, 'html.parser')
  post = soup.find_all('div', {'class':"article-item"})
  all_links = []
  for news in post:
    link = news.find('a', class_='article-item-link').get('href')
    all_links.append(link)
  host = "https://www.moscowzoo.ru" # нам нужно отдельно задать адрес сайта
  for link in all_links:
    whole_link = host + link
    print(whole_link)
    news_item = requests.get(whole_link, headers={'User-Agent': user_agent}, verify = False)
    soup = BeautifulSoup(news_item.text, 'html.parser')
    header = soup.select('cp-banner > template')[0].get_text()
    headers_news_2_pages[header] = soup.find('div', {'class':"dp-content-inner"}).get_text()

In [68]:
for key, value in headers_news_2_pages.items():
  headers_news_2_pages[key] = re.sub('[\n\r\t]', '', value).strip()

In [ ]:
headers_news_2_pages

Для красоты представим это в виде датафрейма.

In [71]:
import pandas as pd

In [93]:
lst1 = list(headers_news_2_pages.keys())
lst2 = list(headers_news_2_pages.values())
df = pd.DataFrame(
    {'Header': lst1,
     'Content': lst2
    })
df

,Header,Content
0,Выставка: Зоопарк в пустыне,В Доме Клюева до 12 ноября проходит выставка и...
1,Остерегайтесь мошенников,
2,С 16 октября Московский зоопарк немного коррек...,Гулять по Зоопарку можно будет с 09:00 до 18:0...
3,В Московском зоопарке поселились марабу,Два аиста марабу приехали в Московский зоопарк...
4,Временно закрыт дополнительный вход у метро Ба...,Уважаемые посетители! По техническим причинам ...
5,Миллионы лет назад: старт нового сезона в Детс...,Детский лекторий Московского зоопарка запускае...
6,Спасём жизнь вместе,
7,Зоопарк переходит на осенний режим работы,"Световой день становится короче, и Московский ..."


In [96]:
import time
import datetime

Бывают сайты, которые, если слишком часто и быстро к ним обращаться, заподазривают неладное и блокируют к ним доступ. Чтобы этого избежать, можно внести намеренную задержку в работу цикла (пару секунд будет достаточно для большинства таких сайтов).

In [97]:
for i in range(4):
  now = datetime.datetime.now()
  date_time = now.strftime("%d/%m/%Y %H:%M:%S")
  print(date_time)
  time.sleep(15)

27/10/2023 11:13:45
27/10/2023 11:14:00
27/10/2023 11:14:15
27/10/2023 11:14:30
